In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
import seaborn as sns
import os, string, random, time, math
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import re
import string
import mlflow
import mlflow.tensorflow

# Device configuration (TensorFlow automatically handles GPU if available)
device_gpu = "/GPU:0" if tf.config.list_physical_devices('GPU') else "/CPU:0"
print(device_gpu)

/CPU:0


In [ ]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU Name:", gpus[0].name)
else:
    print("No GPU found.")


Num GPUs Available: 0
No GPU found.


### Encoding

In [ ]:
eng_alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
pad = '-PAD-'
engdic = {pad : 0}
for index ,alpha in enumerate(eng_alphabets):
  engdic[alpha] = index+1
print(engdic)

{'-PAD-': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}


In [ ]:
#Telugu unicode for hex range is 3072:3199
tel_alphabets = [chr(alpha) for alpha in range(3072,3199)]
tel_alphabets.append(chr(8204))
tel_alphabets.append(chr(8205))
tel_size = len(tel_alphabets)
teldic = {pad : 0}
for index ,alpha in enumerate(tel_alphabets):
  teldic[alpha] = index+1
print(teldic)


{'-PAD-': 0, 'ఀ': 1, 'ఁ': 2, 'ం': 3, 'ః': 4, 'ఄ': 5, 'అ': 6, 'ఆ': 7, 'ఇ': 8, 'ఈ': 9, 'ఉ': 10, 'ఊ': 11, 'ఋ': 12, 'ఌ': 13, '\u0c0d': 14, 'ఎ': 15, 'ఏ': 16, 'ఐ': 17, '\u0c11': 18, 'ఒ': 19, 'ఓ': 20, 'ఔ': 21, 'క': 22, 'ఖ': 23, 'గ': 24, 'ఘ': 25, 'ఙ': 26, 'చ': 27, 'ఛ': 28, 'జ': 29, 'ఝ': 30, 'ఞ': 31, 'ట': 32, 'ఠ': 33, 'డ': 34, 'ఢ': 35, 'ణ': 36, 'త': 37, 'థ': 38, 'ద': 39, 'ధ': 40, 'న': 41, '\u0c29': 42, 'ప': 43, 'ఫ': 44, 'బ': 45, 'భ': 46, 'మ': 47, 'య': 48, 'ర': 49, 'ఱ': 50, 'ల': 51, 'ళ': 52, 'ఴ': 53, 'వ': 54, 'శ': 55, 'ష': 56, 'స': 57, 'హ': 58, '\u0c3a': 59, '\u0c3b': 60, '఼': 61, 'ఽ': 62, 'ా': 63, 'ి': 64, 'ీ': 65, 'ు': 66, 'ూ': 67, 'ృ': 68, 'ౄ': 69, '\u0c45': 70, 'ె': 71, 'ే': 72, 'ై': 73, '\u0c49': 74, 'ొ': 75, 'ో': 76, 'ౌ': 77, '్': 78, '\u0c4e': 79, '\u0c4f': 80, '\u0c50': 81, '\u0c51': 82, '\u0c52': 83, '\u0c53': 84, '\u0c54': 85, 'ౕ': 86, 'ౖ': 87, '\u0c57': 88, 'ౘ': 89, 'ౙ': 90, 'ౚ': 91, '\u0c5b': 92, '\u0c5c': 93, 'ౝ': 94, '\u0c5e': 95, '\u0c5f': 96, 'ౠ': 97, 'ౡ': 98, 'ౢ': 99, 'ౣ': 100, 

In [ ]:
print(tel_alphabets[6]+tel_alphabets[2])

ఆం


### preprocessing

In [ ]:
X=[]
Y=[]
with open(r"EngToTel.txt",encoding = "utf8") as f:
  for line in f:
    line = line.split()
    eng = line[0].strip().upper()
    tel = line[1].strip()
    X.append(eng)
    Y.append(tel)

In [ ]:
print(X[1])
print(Y[1])

DEVATA
దేవత


In [ ]:
def checkAdjChar(instring):
    for i in range(0, len(instring)-2):
        if (instring[i] == instring[i+1]):
            return 1
        else:
            continue
    return 0


In [ ]:
L=[]
M=[]
with open(r"TelToEng.txt",encoding = "utf8") as k:
    for line in k:
        line = line.split()
        EN = line[1].strip().upper()
        TE = line[0].strip()
        if(len(TE)>10 and checkAdjChar(EN)==0):
            L.append(EN)
            M.append(TE)

In [ ]:
print(L[5])
print(M[5])

ANUMATINCHALEDU
అనుమతించలేదు


In [ ]:
X = X+L
Y = Y+M

In [ ]:
print(len(X))

16026


In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [ ]:
print(f"Training size:",len(X_train))
print(f"Test size:",len(X_test))

Training size: 12820
Test size: 3206


In [ ]:
for i in range(3):
  print(X_train[i],Y_train[i])

KUBJA కుబ్జ
MAST మస్త్
KALALU కలలు


### **dataloading**

In [ ]:
class TFDataloader:
    def __init__(self, a, b):
        self.tel_words, self.eng_words = a, b
        self.shuffle_indices = list(range(len(self.tel_words)))
        random.shuffle(self.shuffle_indices)
        self.shuffle_start_index = 0

    def __len__(self):
        return len(self.tel_words)

    def __getitem__(self, idx):
        return self.tel_words[idx], self.eng_words[idx]

    def get_random_sample(self):
        idx = np.random.randint(len(self.tel_words))
        return self.__getitem__(idx)

    def get_batch_from_array(self, batch_size, array):
        end = self.shuffle_start_index + batch_size
        batch = []
        if end >= len(self.tel_words):
            batch = [array[i] for i in self.shuffle_indices[0:end % len(self.tel_words)]]
            end = len(self.tel_words)
        return batch + [array[i] for i in self.shuffle_indices[self.shuffle_start_index:end]]

    def get_batch(self, batch_size, postprocess=True):
        tel_batch = self.get_batch_from_array(batch_size, self.tel_words)
        eng_batch = self.get_batch_from_array(batch_size, self.eng_words)
        self.shuffle_start_index += batch_size

        if self.shuffle_start_index >= len(self.tel_words):
            random.shuffle(self.shuffle_indices)
            self.shuffle_start_index = 0

        return tel_batch, eng_batch

    def to_tf_dataset(self, batch_size):
        """Convert to tf.data.Dataset for TensorFlow training"""
        def batch_generator():
            while True:
                tel_batch, eng_batch = self.get_batch(batch_size)
                for tel, eng in zip(tel_batch, eng_batch):
                    yield tel, eng

        dataset = tf.data.Dataset.from_generator(
            batch_generator,
            output_signature=(
                tf.TensorSpec(shape=(), dtype=tf.string),
                tf.TensorSpec(shape=(), dtype=tf.string)
            )
        )
        return dataset.batch(batch_size)


In [ ]:
 # Create dataloaders exactly like PyTorch
train_data = TFDataloader(Y_train, X_train)
test_data = TFDataloader(Y_test, X_test)

print(type(test_data))

    # This will produce the exact same output as your PyTorch version
for i in range(3):
    print(test_data[i])

<class '__main__.TFDataloader'>
('వడ్లమూరు', 'VADLAMURU')
('రాగంగళ్', 'RAAGANGAL')
('కళ్యాణదుర్గం', 'KALYANDURG')


In [ ]:
bss=train_data.get_batch(10)
print(bss)

(['రాఫెల్', 'ఉప్పుగుండూరు', 'తిరునీర్మలై', 'గుర్రం', 'ప్లెసిస్', 'పుష్కరాలు', 'నెదర్లాండ్స్', 'కుక్క', 'డేల్', 'డీన్'], ['RAFAEL', 'UPPUGUNDUR', 'THIRUNEERMALAI', 'GURRAM', 'PLESSIS', 'PUSHKARAM', 'NETHERLANDS', 'KUKKA', 'DALE', 'DEAN'])


In [ ]:
def word_rep_tf(word, dic):
    word = word.upper()
    vocab_size = len(dic)
    seq_len = len(word) + 1
    one_hot = tf.zeros((seq_len, vocab_size), dtype=tf.float32)

    for i, letter in enumerate(word):
        pos = dic[letter]
        one_hot = tf.tensor_scatter_nd_update(one_hot, [[i, pos]], [1.0])

    pad_pos = dic[pad]
    one_hot = tf.tensor_scatter_nd_update(one_hot, [[len(word), pad_pos]], [1.0])
    return tf.expand_dims(one_hot, axis=0)  # Shape: (1, seq_len, vocab_size)


In [ ]:
def gt_rep(word, dic, device='/CPU:0'):
    word = word.upper()
    with tf.device(device):
        rep = tf.zeros([len(word) + 1, 1], dtype=tf.int64)

        indices = []
        updates = []

        for letter_index, letter in enumerate(word):
            pos = dic[letter]
            indices.append([letter_index, 0])
            updates.append(pos)

        # Add padding at the end
        indices.append([len(word), 0])
        updates.append(dic[pad])

        rep = tf.tensor_scatter_nd_update(rep, indices=indices, updates=updates)
        rep = tf.reshape(rep, (1, -1))

    return rep

In [ ]:
for i in range(3):
    print(Y[i])

పంజాబీ
దేవత
బెషెవిస్


In [ ]:
a=word_rep_tf(Y[0],teldic)

print(a.shape)

(1, 7, 130)


In [ ]:
tel_batch, eng_batch = train_data.get_batch(32)
for i in range(len(tel_batch)):
  print(tel_batch[i],eng_batch[i])
  input_seq = word_rep_tf(tel_batch[i], teldic)  # shape: (1, seq_len, tel_vocab_size)
  gt_seq = gt_rep(eng_batch[i], engdic)
  print(input_seq.shape)
  print(gt_seq.shape)

హొరనాడు HORNADU
(1, 8, 130)
(1, 8)
వీరులపాడు VEERULLAPADU
(1, 10, 130)
(1, 13)
అవెంజర్స్ AVENGERS
(1, 10, 130)
(1, 9)
వ్యవహరించేవారు VYAVAHARINCHAEVARU
(1, 15, 130)
(1, 19)
లాప్లాస్ LAPLACES
(1, 9, 130)
(1, 9)
సిటీస్ CITIES
(1, 7, 130)
(1, 7)
బడేవారిపాలెం BADEVARIPALEM
(1, 13, 130)
(1, 14)
రామాయణ్ RAMAYAN
(1, 8, 130)
(1, 8)
రేస్క్యూయర్స్ RESCUERS
(1, 14, 130)
(1, 9)
విన్స్ VINCE
(1, 7, 130)
(1, 6)
అగర్వాల్ AGGARWAL
(1, 9, 130)
(1, 9)
బావుంటుందని BAVUNTUNDANI
(1, 12, 130)
(1, 13)
నయాపూల్ NAYAPUL
(1, 8, 130)
(1, 8)
గాలికొదిలేశారంటూ GALIKODILESARANTU
(1, 17, 130)
(1, 18)
మెటల్ METAL
(1, 6, 130)
(1, 6)
లోరియస్ LORIUS
(1, 8, 130)
(1, 7)
కొనసాగిందింది KONASAGINDINDI
(1, 14, 130)
(1, 15)
చిన్న CHINNA
(1, 6, 130)
(1, 7)
శిరోద్కర్ SHIRODKAR
(1, 10, 130)
(1, 10)
లియొనార్డో LEONARDO
(1, 11, 130)
(1, 9)
పర్యవసానాలను PARYAVASANALANU
(1, 13, 130)
(1, 16)
ఆజ్ఞాపించిన AGNYAPINCHINA
(1, 12, 130)
(1, 14)
నరస NARASA
(1, 4, 130)
(1, 7)
ధామ్‌ DHAM
(1, 6, 130)
(1, 5)
ప్రవేశించటానికి PRAVESINCHATANIKI
(1, 16

In [ ]:
sample2 = gt_rep(X[0],engdic)
print(X[0])
print(sample2)

PUNJABI
tf.Tensor([[16 21 14 10  1  2  9  0]], shape=(1, 8), dtype=int64)


### **GRU With Attention**

In [ ]:
class TransliterationEncoderDecoder(tf.keras.Model):
    def __init__(self, input_size, hidden_size, output_size):
        super(TransliterationEncoderDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.encoder_rnn_cell = tf.keras.layers.GRU(hidden_size, return_sequences=True, return_state=True)
        self.decoder_rnn_cell = tf.keras.layers.GRU(hidden_size, return_state=True)

        self.h2o = tf.keras.layers.Dense(output_size, activation='softmax')
        self.U = tf.keras.layers.Dense(hidden_size)
        self.W = tf.keras.layers.Dense(hidden_size)
        self.attn = tf.keras.layers.Dense(1)
        self.out2hidden = tf.keras.layers.Dense(hidden_size)

    def call(self, input_, max_char=30, gt=None,training=True):
        batch_size = tf.shape(input_)[0]

        # Encoder
        eout, hidden = self.encoder_rnn_cell(input_)
        #print("eout shape:", eout.shape)
        #print("encoder final hidden state shape:", hidden.shape)

        decoder_state = hidden
        decoder_state = tf.reshape(decoder_state, (1,-1))

        decoder_input = tf.zeros((batch_size, 1, self.output_size))  # Start with <SOS> one-hot
        #print("initial decoder input shape:", decoder_input.shape)

        outputs = []

        U = self.U(eout)
        #print("U shape:", U.shape)

        for i in range(max_char):
            W = self.W(decoder_state)
            #print(f"\nTime Step {i+1}:")
            #print("W (before expand) shape:", W.shape)
            W = tf.expand_dims(W, 1)
            #print("W shape:", W.shape)

            attn_input = tf.nn.tanh(U + W)
            #print("attention input shape (U + W):", attn_input.shape)

            V = self.attn(attn_input)
            #print("V shape:", V.shape)

            attn_weights = tf.nn.softmax(V, axis=1)
            #print("attention weights shape:", attn_weights.shape)

            attn_applied = tf.reduce_sum(attn_weights * eout, axis=1, keepdims=True)
            #print("attention applied shape:", attn_applied.shape)

            embedded = self.out2hidden(decoder_input)
            #print("embedded decoder input shape:", embedded.shape)

            decoder_input_concat = tf.concat([embedded, attn_applied], axis=-1)
            #print("concatenated decoder input shape:", decoder_input_concat.shape)

            out, decoder_state = self.decoder_rnn_cell(decoder_input_concat, initial_state=decoder_state)
            decoder_state = tf.reshape(decoder_state, (1,-1))
            #print("decoder GRU output shape:", out.shape)
            #print("decoder new state shape:", decoder_state.shape)

            out = self.h2o(tf.expand_dims(out, axis=1))
            #print("output after final dense + softmax shape:", out.shape)

            outputs.append(out[:, 0, :])
            #print("appended output shape:", outputs[-1].shape)
            # gt = tf.reshape(gt, (batch_size, max_char))
            # print(gt.shape)
            # print(gt)
            idx = tf.argmax(out, axis=2, output_type=tf.int32)

            if training and gt is not None:
                # Use ground truth index at timestep `i`
                idx = tf.expand_dims(gt[:, i], axis=1)
            # One-hot encode
            one_hot = tf.one_hot(idx, depth=self.output_size)

            # Detach (not needed in TF for one-hot because it's non-trainable input, but for symmetry):
            decoder_input = tf.stop_gradient(one_hot)

        final_output = tf.stack(outputs, axis=1)
        #print("final stacked output shape:", final_output.shape)
        return final_output






In [ ]:
net = TransliterationEncoderDecoder(len(teldic),256,len(engdic))


In [ ]:
def infer(model, name, n, device='/CPU:0'):
    with tf.device(device):
        # word_rep_tf now returns [1, seq_len, vocab_size]
        name_ohe = word_rep_tf(name, teldic)

        # Call the model
        # Model returns [max_char, batch_size, output_size]
        output_batch = model(name_ohe, max_char=n,training=False)

        # Assuming batch size is 1 for inference, take the first (and only) element
        # output_single shape: [max_char, output_size]
        output_single = output_batch[:, 0, :]

        return output_single

### **Training**

In [ ]:
def train_batch_tf(model, optimizer, loss_fn, tel_batch, eng_batch, teldic, engdic, max_char, device="/CPU:0"):
    with tf.device(device):
        batch_size = len(tel_batch)
        total_loss = 0

        for i in range(batch_size):
            input_seq = word_rep_tf(tel_batch[i], teldic)  # shape: (1, seq_len, tel_vocab_size)
            gt_seq = gt_rep(eng_batch[i], engdic)
            gt_without_squeze=gt_seq     # shape: (1, target_len)
            # print(gt_seq)
            # Squeeze the gt_seq to remove the batch dimension
            gt_seq = tf.squeeze(gt_seq, axis=0)  # shape: (target_len,)
            # print(gt_seq)

            # print("inputshape",input_seq.shape)
            # print("gtshape:",gt_seq.shape)

            max_char = tf.shape(gt_seq)[0]
            with tf.GradientTape() as tape:
                output = model(input_seq, max_char=max_char,gt=gt_without_squeze)  # shape: (1, max_char, eng_vocab_size)
                # print(output)
                output = output[0]  # shape: (max_char, eng_vocab_size)

                loss = 0

                for t in range(tf.shape(gt_seq)[0]):
                    token_loss = loss_fn(gt_seq[t], output[t])  # Compare each ground truth token with predicted logits
                    loss += token_loss
                loss = loss / tf.cast(tf.shape(gt_seq)[0], tf.float32)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            total_loss += loss.numpy()

        return total_loss / batch_size


In [ ]:
def train_setup_tf(model, optimizer, loss_fn, dataset, teldic, engdic, n_batches=100, batch_size=10, max_char=20, device="/CPU:0"):
    loss_arr = []

    for i in range(n_batches):
        tel_batch, eng_batch = dataset.get_batch(batch_size)
        #print("hi in tsetip")
        loss = train_batch_tf(model, optimizer, loss_fn, tel_batch, eng_batch, teldic, engdic, max_char, device=device)
        loss_arr.append(loss)

        if True:
            print(f"Iteration {i}, Loss: {loss:.4f}")
            plt.plot(loss_arr, '-*')
            plt.xlabel("Iteration")
            plt.ylabel("Loss")
            plt.show()
            clear_output(wait=True)

    return loss_arr


In [ ]:
def calc_accuracy_tf(model, device='/CPU:0'):
    predictions = []
    accuracy = 0.0

    with tf.device(device):
        for i in range(len(test_data)):
            tel, eng = test_data[i]
            gt = gt_rep(eng, engdic, device)  # Shape: (seq_len, 1)

            outputs = infer(model, tel, gt.shape[0], device=device)  # List of tensors, shape [1, 1, vocab_size] each
            correct = 0

            for index, out in enumerate(outputs):
                # Adjust indexing to handle potential scalar or [1,1] shape from argmax
                pred_index = tf.argmax(out, axis=-1).numpy().flatten()[0]  # Predicted class
                true_index = gt[index][0].numpy()                   # Ground truth class

                if pred_index == true_index:
                    correct += 1

            accuracy += correct / gt.shape[0]

        accuracy /= len(test_data)
        accuracy *= 100

        # mlflow.log_metric('test_accuracy', accuracy) # Commenting out MLflow for now
        print(f"Test Accuracy: {accuracy:.2f}%")
        return accuracy

In [ ]:
# calc_accuracy_tf(model=model_tf)

In [ ]:
# import tensorflow as tf
# import numpy as np
# import os

best_accuracy = -1.0
best_model_path = "best_model_tf.weights.h5"
  # Create a new model
model = TransliterationEncoderDecoder(len(teldic), 256, len(engdic))
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False,reduction=None)
for lr in [0.001]:
    for batch_size in [32, 64]:
        for n_batches in [100, 150, 200]:
            expt_id = f"{int(lr*100)}_{n_batches}_{batch_size}"

            # Train model
            train_setup_tf(model, optimizer,loss_fn,train_data,teldic,engdic, n_batches, batch_size, max_char=30, device=device_gpu)

            # Evaluate model
            current_accuracy = calc_accuracy_tf(model, device=device_gpu)

            # Check and save the best model
            if current_accuracy > best_accuracy:
                best_accuracy = current_accuracy

                print(f"New best model saved with accuracy: {best_accuracy:.2f}%")

            # Clear memory
            tf.keras.backend.clear_session()


Test Accuracy: 92.11%


TypeError: '>' not supported between instances of 'NoneType' and 'float'

In [ ]:
print(model.summary())

Model: "transliteration_encoder_decoder_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_6 (GRU)                     │ ((1, 6, 256), (1,      │       297,984 │
│                                 │ 256))                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_7 (GRU)                     │ ((1, 256), (1, 256))   │       591,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (1, 1, 27)             │         6,939 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (1, 6, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (1, 256)               │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (1, 6, 1)              │           257 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (1, 1, 256)            │         7,168 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,035,292 (3.95 MB)

 Trainable params: 1,035,292 (3.95 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
def test_one_example(net, tel_input, tel_dic, eng_dic, max_char=30):
    rev_eng_dic = {v: k for k, v in eng_dic.items()}

    input_seq = word_rep_tf(tel_input, tel_dic)  # shape: (seq_len, input_size)
    # input_seq = tf.expand_dims(input_seq, axis=0)  # shape: (1, seq_len, input_size)

    output = net(input_seq, max_char=max_char, training=False)  # shape: (1, max_char, vocab_size)

    pred_indices = tf.argmax(output, axis=2)[0].numpy()  # (max_char,)
    pred_chars = [idx for idx in pred_indices if idx != eng_dic['-PAD-']]  # Assuming you have '<pad>' in vocab
    decoded = ''.join([rev_eng_dic[i] for i in pred_chars])

    print(f"Input: {tel_input} -> Predicted Output: {decoded}")


In [ ]:
test_one_example(model, "ప్రతిబింబించటానికి", teldic, engdic, max_char=30)


Input: ప్రతిబింబించటానికి -> Predicted Output: PRATIBINCHANICHANIINCHINCH
